## Autoencoders - classification (mnist dataset)

In [60]:
import tensorflow as tf
import keras
import numpy as np

In [61]:
tf.__version__, keras.__version__, np.__version__

('2.17.0', '3.5.0', '1.26.4')

In [62]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import InputLayer, Input, Dense
from tensorflow.keras import utils as np_utils

In [63]:
(X_train, y_train),(X_test, y_test) = mnist.load_data()

In [64]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [65]:
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [66]:
y_train = np_utils.to_categorical(y_train)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [67]:
y_test = np_utils.to_categorical(y_test)

In [68]:
X_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))
X_train.shape, X_test.shape

((60000, 784), (10000, 784))

In [69]:
# 784 -> 32 -> 784

autoencoder = Sequential()
autoencoder.add(InputLayer(shape=(784,)))
autoencoder.add(Dense(units=32, activation='relu'))
autoencoder.add(Dense(units=784, activation='sigmoid'))
autoencoder.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 32)             │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 784)            │        25,872 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,992 (199.19 KB)

 Trainable params: 50,992 (199.19 KB)

 Non-trainable params: 0 (0.00 B)

In [70]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])


In [71]:
autoencoder.fit(X_train, X_train, epochs = 50,
                batch_size=256, validation_data=(X_test, X_test))

Epoch 1/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0092 - loss: 0.3858 - val_accuracy: 0.0119 - val_loss: 0.1887
Epoch 2/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0121 - loss: 0.1780 - val_accuracy: 0.0101 - val_loss: 0.1526
Epoch 3/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0119 - loss: 0.1486 - val_accuracy: 0.0097 - val_loss: 0.1338
Epoch 4/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0120 - loss: 0.1316 - val_accuracy: 0.0093 - val_loss: 0.1212
Epoch 5/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0117 - loss: 0.1205 - val_accuracy: 0.0099 - val_loss: 0.1128
Epoch 6/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0098 - loss: 0.1126 - val_accuracy: 0.0101 - val_loss: 0.1068
Epoch 7/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0107 - loss: 0.1071 - val_accuracy: 0.0121 - val_loss: 0.1026
Epoch 8/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0110 - loss: 0.1030 - val_accuracy: 0.

In [72]:
autoencoder.layers

[<Dense name=dense_21, built=True>, <Dense name=dense_22, built=True>]

In [73]:
or_dim = Input(shape=(784,))
encoder_layer = autoencoder.layers[0]
encoder = Model(or_dim, encoder_layer(or_dim))

In [74]:
encoded_train = encoder.predict(X_train)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 503us/step


In [75]:
encoded_test = encoder.predict(X_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step


In [76]:
encoded_train.shape, encoded_test.shape

((60000, 32), (10000, 32))

Classification itself without dimentionality reduction

In [77]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 784), (60000, 10), (10000, 784), (10000, 10))

In [78]:
# 784 (input neurons) + 10 (output neurons) / 2 = 397
# 784 -> 397 -> 397 -> 10
c1 = Sequential()
c1.add(InputLayer(shape = (784,)))
c1.add(Dense(units = 397, activation='relu'))
c1.add(Dense(units = 397, activation='relu'))
c1.add(Dense(units = 10, activation='softmax'))

c1.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

c1.fit(X_train, y_train, batch_size=256, epochs = 100,
       validation_data=(X_test, y_test))

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8574 - loss: 0.5219 - val_accuracy: 0.9626 - val_loss: 0.1252
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9680 - loss: 0.1049 - val_accuracy: 0.9700 - val_loss: 0.0963
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9811 - loss: 0.0638 - val_accuracy: 0.9750 - val_loss: 0.0759
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9877 - loss: 0.0405 - val_accuracy: 0.9757 - val_loss: 0.0763
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9907 - loss: 0.0295 - val_accuracy: 0.9801 - val_loss: 0.0649
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9929 - loss: 0.0222 - val_accuracy: 0.9790 - val_loss: 0.0669
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9953 - loss: 0.0160 - val_accuracy: 0.9807 - val_loss: 0.0735
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9957 - loss: 0.0138 - val_accu

Classification itself with dimentionality reduction (it is faster to train)

In [79]:
# (32 + 10)/ 2 = 
# # 32 -> 21 -> 21 -> 10
c2 = Sequential()
c2.add(InputLayer(shape = (32,)))
c2.add(Dense(units = 21, activation='relu'))
c2.add(Dense(units = 21, activation='relu'))
c2.add(Dense(units = 10, activation='softmax'))

c2.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

c2.fit(encoded_train, y_train, batch_size=256, epochs = 100,
       validation_data=(encoded_test, y_test))

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2320 - loss: 4.7918 - val_accuracy: 0.6725 - val_loss: 1.0215
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7113 - loss: 0.9018 - val_accuracy: 0.8089 - val_loss: 0.6128
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8125 - loss: 0.6032 - val_accuracy: 0.8542 - val_loss: 0.4707
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8552 - loss: 0.4739 - val_accuracy: 0.8757 - val_loss: 0.4016
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8760 - loss: 0.4053 - val_accuracy: 0.8914 - val_loss: 0.3515
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8875 - loss: 0.3662 - val_accuracy: 0.8965 - val_loss: 0.3315
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8955 - loss: 0.3416 - val_accuracy: 0.9083 - val_loss: 0.3027
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9030 - loss: 0.3213 - val_accu